In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler


# PREPROCESAMIENTO DEL CONJUNTO DE PRUEBA

Se le aplican las mismas transformaciones que al conjunto de entrenamiento, salvo casos muy concretos relacionados con datos ouliers en los que no sería correcto hacerlo.

In [2]:
data= pd.read_csv("Data/test.csv")


## MSSubClass

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

In [3]:
data['MSSubClass'] = data['MSSubClass'].replace({
    20: '1-STORY',
    30: '1-STORY',
    40: '1-STORY',
    120: '1-STORY',
    45: '1.5-STORY',
    50: '1.5-STORY',
    150: '1.5-STORY',
    60: '2-STORY',
    70: '2-STORY',
    160: '2-STORY',
    75: '2.5-STORY',
    80: 'SPLIT/MULTI-LEVEL',
    85: 'SPLIT/MULTI-LEVEL',
    180: 'SPLIT/MULTI-LEVEL',
    90: 'DUPLEX',
    190: '2 FAMILY CONVERSION'
})

cleaned_data = data.copy()
ms_subclass_encoded = pd.get_dummies(cleaned_data['MSSubClass'], prefix='MSSubClass', drop_first=True)
cleaned_data = cleaned_data.drop('MSSubClass', axis=1)
cleaned_data = pd.concat([cleaned_data, ms_subclass_encoded], axis=1)



## MSZoning

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density

In [4]:
mszoning_encoded = pd.get_dummies(cleaned_data['MSZoning'], prefix='MSZoning', drop_first=True)
cleaned_data = pd.concat([cleaned_data, mszoning_encoded], axis=1)
cleaned_data = cleaned_data.drop('MSZoning', axis=1)


## LotFrontage

LotFrontage: Linear feet of street connected to property

In [5]:
cleaned_data['LotFrontage'].fillna(0, inplace=True)


## LotArea


Lot size in square feet

In [6]:
cleaned_data['LotArea'] = np.log1p(cleaned_data['LotArea'])

## Street


Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved

In [7]:
cleaned_data = cleaned_data.drop('Street', axis=1)

## Alley

Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access

In [8]:
cleaned_data = cleaned_data.drop('Alley', axis=1)

## LotShape

LotShape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular
       

In [9]:
cleaned_data['LotShape'] = cleaned_data['LotShape'].replace({'IR1': 'IR', 'IR2': 'IR', 'IR3': 'IR'})
lotshape_encoded = pd.get_dummies(cleaned_data['LotShape'], prefix='LotShape', drop_first=True)
cleaned_data = cleaned_data.drop('LotShape', axis=1)
cleaned_data = pd.concat([cleaned_data, lotshape_encoded], axis=1)

 ## LandContour


LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression

In [10]:
cleaned_data['LandContour'] = cleaned_data['LandContour'].replace({'Lvl': 'Lvl', 'Bnk': 'No Lvl', 'HLS': 'No Lvl', 'Low': 'No Lvl'})
landcontour_encoded = pd.get_dummies(cleaned_data['LandContour'], prefix='LandContour', drop_first=True)
cleaned_data = cleaned_data.drop('LandContour', axis=1)
cleaned_data = pd.concat([cleaned_data, landcontour_encoded], axis=1)

## Utilities

Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only	

In [11]:
cleaned_data = cleaned_data.drop('Utilities', axis=1)

## LotConfig

LotConfig: Lot configuration

       Inside	Inside lot
       Corner	Corner lot
       CulDSac	Cul-de-sac
       FR2	Frontage on 2 sides of property
       FR3	Frontage on 3 sides of property

In [12]:
lotconfig_encoded = pd.get_dummies(cleaned_data['LotConfig'], prefix='LotConfig', drop_first=True)
cleaned_data = cleaned_data.drop('LotConfig', axis=1)
cleaned_data = pd.concat([cleaned_data, lotconfig_encoded], axis=1)


## LandsLope

LandSlope: Slope of property
		
       Gtl	Gentle slope
       Mod	Moderate Slope	
       Sev	Severe Slope

In [13]:
cleaned_data['LandSlope'] = cleaned_data['LandSlope'].replace({'Gtl': 'Gtl', 'Mod': 'No Gtl', 'Sev': 'No Gtl'})
landslope_encoded = pd.get_dummies(cleaned_data['LandSlope'], prefix='LandSlope', drop_first=True)
cleaned_data = cleaned_data.drop('LandSlope', axis=1)
cleaned_data = pd.concat([cleaned_data, landslope_encoded], axis=1)


## Neighborhood

Neighborhood: Physical locations within Ames city limits

       Blmngtn	Bloomington Heights
       Blueste	Bluestem
       BrDale	Briardale
       BrkSide	Brookside
       ClearCr	Clear Creek
       CollgCr	College Creek
       Crawfor	Crawford
       Edwards	Edwards
       Gilbert	Gilbert
       IDOTRR	Iowa DOT and Rail Road
       MeadowV	Meadow Village
       Mitchel	Mitchell
       Names	North Ames
       NoRidge	Northridge
       NPkVill	Northpark Villa
       NridgHt	Northridge Heights
       NWAmes	Northwest Ames
       OldTown	Old Town
       SWISU	South & West of Iowa State University
       Sawyer	Sawyer
       SawyerW	Sawyer West
       Somerst	Somerset
       StoneBr	Stone Brook
       Timber	Timberland
       Veenker	Veenker

In [14]:
neighborhood_dict = {'Blmngtn': 'MildPrice',
 'Blueste': 'MildPrice',
 'BrDale': 'LowPrice',
 'BrkSide': 'LowPrice',
 'ClearCr': 'MildPrice',
 'CollgCr': 'MildPrice',
 'Crawfor': 'MildPrice',
 'Edwards': 'LowPrice',
 'Gilbert': 'MildPrice',
 'IDOTRR': 'LowPrice',
 'MeadowV': 'LowPrice',
 'Mitchel': 'MildPrice',
 'NAmes': 'MildPrice',
 'NPkVill': 'MildPrice',
 'NWAmes': 'MildPrice',
 'NoRidge': 'HighPrice',
 'NridgHt': 'HighPrice',
 'OldTown': 'LowPrice',
 'SWISU': 'MildPrice',
 'Sawyer': 'MildPrice',
 'SawyerW': 'MildPrice',
 'Somerst': 'HighPrice',
 'StoneBr': 'HighPrice',
 'Timber': 'HighPrice',
 'Veenker': 'HighPrice'}


In [15]:
cleaned_data['Neighborhood'] = cleaned_data['Neighborhood'].map(neighborhood_dict)
cleaned_data.groupby('Neighborhood')['Neighborhood'].count()

Neighborhood
HighPrice    288
LowPrice     360
MildPrice    811
Name: Neighborhood, dtype: int64

In [16]:
price_order = {'HighPrice': 2, 'MildPrice': 1, 'LowPrice': 0}
cleaned_data['Neighborhood'] = cleaned_data['Neighborhood'].map(price_order)

## Condition 1 y 2

Condition1: Proximity to various conditions
	
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
Condition2: Proximity to various conditions (if more than one is present)
		
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad

In [17]:
cleaned_data['Condition1'] = cleaned_data['Condition1'].replace({
    'Norm': 'Normal',
    'Artery': 'Street',
    'Feedr': 'Street',
    'RRNn': 'Railroad',
    'RRAn': 'Railroad',
    'PosN': 'Positive Feature',
    'PosA': 'Positive Feature',
    'RRNe': 'Railroad',
    'RRAe': 'Railroad'
})

cleaned_data['Condition2'] = cleaned_data['Condition2'].replace({
    'Norm': 'Normal',
    'Artery': 'Street',
    'Feedr': 'Street',
    'RRNn': 'Railroad',
    'RRAn': 'Railroad',
    'PosN': 'Positive Feature',
    'PosA': 'Positive Feature',
    'RRNe': 'Railroad',
    'RRAe': 'Railroad'
})

condition1_encoded = pd.get_dummies(cleaned_data['Condition1'], prefix='Condition1', drop_first=True)
condition2_encoded = pd.get_dummies(cleaned_data['Condition2'], prefix='Condition2', drop_first=True)

cleaned_data['Street_Condition'] = condition1_encoded.get('Condition1_Street', 0) + condition2_encoded.get('Condition2_Street', 0)
cleaned_data['Railroad_Condition'] = condition1_encoded.get('Condition1_Railroad', 0) + condition2_encoded.get('Condition2_Railroad', 0)
cleaned_data['PositiveFeature_Condition'] = condition1_encoded.get('Condition1_Positive Feature', 0) + condition2_encoded.get('Condition2_Positive Feature', 0)

cleaned_data = cleaned_data.drop(columns=['Condition1'])
cleaned_data = cleaned_data.drop(columns=['Condition2'])












## BldgType

BldgType: Type of dwelling
		
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit

In [18]:
cleaned_data['BldgGroup'] = cleaned_data['BldgType'].replace({
    '1Fam': 'Vivienda Individual',
    'TwnhsE': 'Vivienda Individual',
    '2fmCon': 'Doble Vivienda',
    'Duplex': 'Doble Vivienda',
    'Twnhs': 'Múltiples Unidades'
})

bldg_group_encoded = pd.get_dummies(cleaned_data['BldgGroup'], prefix='BldgGroup')
cleaned_data = pd.concat([cleaned_data, bldg_group_encoded], axis=1)
cleaned_data = cleaned_data.drop('BldgGroup', axis=1)
cleaned_data = cleaned_data.drop(columns=['BldgType'])


## HouseStyle

HouseStyle: Style of dwelling
	
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level

In [19]:
cleaned_data = cleaned_data.drop('HouseStyle', axis=1)


## OverallQual

OverallQual: Rates the overall material and finish of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor

## OverallCond

OverallCond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor

 ## YearBuilt

YearBuilt: Original construction date

In [20]:
current_year = 2024
cleaned_data['Age'] = current_year - cleaned_data['YearBuilt']

cleaned_data = cleaned_data.drop('YearBuilt', axis=1)

## YearRemodAdd

YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)


In [21]:
current_year = 2024
cleaned_data['Ageremod'] = current_year - cleaned_data['YearRemodAdd']

cleaned_data = cleaned_data.drop('YearRemodAdd', axis=1)

## RoofStyle

RoofStyle: Type of roof

       Flat	Flat
       Gable	Gable
       Gambrel	Gabrel (Barn)
       Hip	Hip
       Mansard	Mansard
       Shed	Shed

In [22]:
data['RoofStyle'] = data['RoofStyle'].replace({
    'Flat': 'Flat_Shed',
    'Shed': 'Flat_Shed',
    'Gable': 'Gable_Hip_Gambrel',
    'Hip': 'Gable_Hip_Gambrel',
    'Gambrel': 'Gable_Hip_Gambrel',
    'Mansard': 'Mansard'
})

roofstyle_encoded = pd.get_dummies(data['RoofStyle'], prefix='RoofStyle', drop_first=True)
data = data.drop('RoofStyle', axis=1)
data = pd.concat([data, roofstyle_encoded], axis=1)
cleaned_data = cleaned_data.drop(columns=['RoofStyle'])







## RoofMatl

RoofMatl: Roof material

       ClyTile	Clay or Tile
       CompShg	Standard (Composite) Shingle
       Membran	Membrane
       Metal	Metal
       Roll	Roll
       Tar&Grv	Gravel & Tar
       WdShake	Wood Shakes
       WdShngl	Wood Shingles

In [23]:
RoofMatl_dict = {'CompShg': 'MildPrice',
 'Membran': 'HighPrice',
 'Metal': 'MildPrice',
 'Roll': 'MildPrice',
 'Tar&Grv': 'MildPrice',
 'WdShake': 'HighPrice',
 'WdShngl': 'HighPrice'}

In [24]:
cleaned_data['RoofMatl'] = cleaned_data['RoofMatl'].map(RoofMatl_dict)
cleaned_data.groupby('RoofMatl')['RoofMatl'].count()

RoofMatl
HighPrice       5
MildPrice    1454
Name: RoofMatl, dtype: int64

In [25]:
price_order = {'HighPrice': 2, 'MildPrice': 1, 'LowPrice': 0}
cleaned_data['RoofMatl'] = cleaned_data['RoofMatl'].map(price_order)

## Exterior1st y exterior2nd

Exterior1st: Exterior covering on house

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast	
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
Exterior2nd: Exterior covering on house (if more than one material)

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles

In [26]:
Exterior1st_dict = {'AsbShng': 'LowPrice',
 'AsphShn': 'LowPrice',
 'BrkComm': 'LowPrice',
 'BrkFace': 'MildPrice',
 'CBlock': 'LowPrice',
 'CemntBd': 'HighPrice',
 'HdBoard': 'MildPrice',
 'ImStucc': 'HighPrice',
 'MetalSd': 'MildPrice',
 'Plywood': 'MildPrice',
 'Stone': 'HighPrice',
 'Stucco': 'MildPrice',
 'VinylSd': 'MildPrice',
 'Wd Sdng': 'MildPrice',
 'WdShing': 'MildPrice'}

Exterior2nd_dict = {'AsbShng': 'LowPrice',
 'AsphShn': 'MildPrice',
 'Brk Cmn': 'LowPrice',
 'BrkFace': 'MildPrice',
 'CBlock': 'LowPrice',
 'CmentBd': 'HighPrice',
 'HdBoard': 'MildPrice',
 'ImStucc': 'HighPrice',
 'MetalSd': 'MildPrice',
 'Other': 'HighPrice',
 'Plywood': 'MildPrice',
 'Stone': 'MildPrice',
 'Stucco': 'MildPrice',
 'VinylSd': 'HighPrice',
 'Wd Sdng': 'MildPrice',
 'Wd Shng': 'MildPrice'}

price_order = {'HighPrice': 2, 'MildPrice': 1, 'LowPrice': 0}

cleaned_data['Exterior1st'] = cleaned_data['Exterior1st'].map(Exterior1st_dict).map(price_order)
cleaned_data['Exterior2nd'] = cleaned_data['Exterior2nd'].map(Exterior2nd_dict).map(price_order)

cleaned_data['ExteriorCombined'] = cleaned_data[['Exterior1st', 'Exterior2nd']].mean(axis=1)
cleaned_data = cleaned_data.drop(columns=['Exterior1st', 'Exterior2nd'])


## MasVnrType

MasVnrType: Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None 	None
       Stone	Stone

In [27]:
cleaned_data['MasVnrType'] = cleaned_data['MasVnrType'].fillna('None')
cleaned_data['MasVnrType'] = cleaned_data['MasVnrType'].replace({
    'None': 'BrkCmn',
})

mas_vnr_dummies = pd.get_dummies(cleaned_data['MasVnrType'], prefix='MasVnrType', drop_first=True)
cleaned_data = cleaned_data.drop('MasVnrType', axis=1)
cleaned_data = pd.concat([cleaned_data, mas_vnr_dummies], axis=1)


## MasVnrArea

MasVnrArea: Masonry veneer area in square feet


In [28]:
cleaned_data['MasVnrArea'].fillna(0, inplace=True)


## ExterQual

ExterQual: Evaluates the quality of the material on the exterior 
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

In [29]:
ExterQual_dict = {
    'Ex': 4,
    'Gd': 3,
    'TA': 2,
    'Fa': 1,
    'Po': 0
}

cleaned_data['ExterQual'] = cleaned_data['ExterQual'].map(ExterQual_dict)


## ExterCond

ExterCond: Evaluates the present condition of the material on the exterior
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		

In [30]:
ExterCond_dict = {
    'Ex': 4,
    'Gd': 3,
    'TA': 2,
    'Fa': 1,
    'Po': 0
}

cleaned_data['ExterCond'] = cleaned_data['ExterCond'].map(ExterCond_dict)

## Foundation

Foundation: Type of foundation
		
       BrkTil	Brick & Tile
       CBlock	Cinder Block
       PConc	Poured Contrete	
       Slab	Slab
       Stone	Stone
       Wood	Wood

In [31]:
foundation_encoded = pd.get_dummies(cleaned_data['Foundation'], prefix='Foundation', drop_first=True)
cleaned_data = cleaned_data.drop('Foundation', axis=1)
cleaned_data = pd.concat([cleaned_data, foundation_encoded], axis=1)


## BsmtQual

BsmtQual: Evaluates the height of the basement

       Ex	Excellent (100+ inches)	
       Gd	Good (90-99 inches)
       TA	Typical (80-89 inches)
       Fa	Fair (70-79 inches)
       Po	Poor (<70 inches
       NA	No Basement

In [32]:
cleaned_data['BsmtQual'].fillna('NA', inplace=True)
bsmt_qual_mapping = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA': 0
}

cleaned_data['BsmtQual'] = cleaned_data['BsmtQual'].map(bsmt_qual_mapping)


In [33]:
cleaned_data['HasBasement'] = cleaned_data['BsmtQual'].apply(lambda x: 1 if x != 'NA' else 0)


## BsmtCond

BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness
       NA	No Basement

In [34]:
cleaned_data['BsmtCond'].fillna('NA', inplace=True)
bsmt_cond_mapping = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA': 0
}

cleaned_data['BsmtCond'] = cleaned_data['BsmtCond'].map(bsmt_cond_mapping)


## BsmtExposure

BsmtExposure: Refers to walkout or garden level walls

       Gd	Good Exposure
       Av	Average Exposure (split levels or foyers typically score average or above)	
       Mn	Mimimum Exposure
       No	No Exposure
       NA	No Basement

In [35]:
cleaned_data['BsmtExposure'].fillna('NA', inplace=True)
cleaned_data['BsmtExposure'] = cleaned_data['BsmtExposure'].replace({
    'Gd': 4,
    'Av': 3,
    'Mn': 2,
    'No': 1,
    'NA': 0
})



BsmtFinSF2: Type 2 finished square feet


## BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF y TotalBsmtSF

BsmtFinType1: Rating of basement finished area

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement
		
BsmtFinSF1: Type 1 finished square feet

BsmtFinType2: Rating of basement finished area (if multiple types)

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement

BsmtFinSF2: Type 2 finished square feet

BsmtUnfSF: Unfinished square feet of basement area

TotalBsmtSF: Total square feet of basement area

In [36]:
cleaned_data['BsmtFinType1'].fillna('NA', inplace=True)
cleaned_data['BsmtFinType2'].fillna('NA', inplace=True)

cleaned_data['TotalBsmtSF_Check'] = (
    cleaned_data['BsmtFinSF1'] +
    cleaned_data['BsmtFinSF2'] +
    cleaned_data['BsmtUnfSF']
)

check_result = (cleaned_data['TotalBsmtSF'] == cleaned_data['TotalBsmtSF_Check'])
check_result.value_counts()

cleaned_data['BsmtFinType1'] = cleaned_data['BsmtFinType1'].replace({
    'GLQ': 6,
    'ALQ': 5,
    'BLQ': 4,
    'Rec': 3,
    'LwQ': 2,
    'Unf': 1,
    'NA': 0
})

cleaned_data['BsmtFinType2'] = cleaned_data['BsmtFinType2'].replace({
    'GLQ': 6,
    'ALQ': 5,
    'BLQ': 4,
    'Rec': 3,
    'LwQ': 2,
    'Unf': 1,
    'NA': 0
})

max_area = cleaned_data['TotalBsmtSF'].max()
cleaned_data['BsmtFinSF1'] = cleaned_data['BsmtFinSF1'] / max_area
cleaned_data['BsmtFinSF2'] = cleaned_data['BsmtFinSF2'] / max_area
cleaned_data['BsmtUnfSF'] = cleaned_data['BsmtUnfSF'] / max_area

cleaned_data['BsmtQualityScore'] = (
    (cleaned_data['BsmtFinType1'] * cleaned_data['BsmtFinSF1']) + 
    (cleaned_data['BsmtFinType2'] * cleaned_data['BsmtFinSF2']) + 
    cleaned_data['BsmtUnfSF']
)

cleaned_data = cleaned_data.drop(columns=['BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotalBsmtSF_Check'])



## Heating

Heating: Type of heating
		
       Floor	Floor Furnace
       GasA	Gas forced warm air furnace
       GasW	Gas hot water or steam heat
       Grav	Gravity furnace	
       OthW	Hot water or steam heat other than gas
       Wall	Wall furnace

In [37]:
cleaned_data['Heating_GasA'] = cleaned_data['Heating'].apply(lambda x: 1 if x == 'GasA' else 0)
cleaned_data = cleaned_data.drop(columns=['Heating'])


## HeatingQC

HeatingQC: Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

In [38]:
cleaned_data['HeatingQC'] = cleaned_data['HeatingQC'].replace({
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
})

## CentralAir

CentralAir: Central air conditioning

       N	No
       Y	Yes
		

In [39]:
cleaned_data['CentralAir'] = cleaned_data['CentralAir'].replace({
    'Y': 1,
    'N': 0
})


## Electrical

Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed

In [40]:
cleaned_data['Electrical_SBrkr'] = cleaned_data['Electrical'].apply(lambda x: 1 if x == 'SBrkr' else 0)
cleaned_data = cleaned_data.drop(columns=['Electrical'])


## 1stFlrSF

1stFlrSF: First Floor square feet


In [41]:
data['1stFlrSF'].describe()

count    1459.000000
mean     1156.534613
std       398.165820
min       407.000000
25%       873.500000
50%      1079.000000
75%      1382.500000
max      5095.000000
Name: 1stFlrSF, dtype: float64

## 2ndFlrSF

2ndFlrSF: Second floor square feet


In [42]:
data['2ndFlrSF'].describe()

count    1459.000000
mean      325.967786
std       420.610226
min         0.000000
25%         0.000000
50%         0.000000
75%       676.000000
max      1862.000000
Name: 2ndFlrSF, dtype: float64

## LowQualFinSF

Low quality finished square feet (all floors)


In [43]:
cleaned_data.drop(columns=['LowQualFinSF'], inplace=True)


## GrLivArea

GrLivArea: Above grade (ground) living area square feet

## BsmtFullBath y BsmtHalfBath

BsmtFullBath: Basement full bathrooms

BsmtHalfBath: Basement half bathrooms

In [44]:
cleaned_data['TotalBasementBaths'] = (
    cleaned_data['BsmtFullBath'] + (cleaned_data['BsmtHalfBath'] * 0.5)
)
cleaned_data = cleaned_data.drop(columns=['BsmtFullBath', 'BsmtHalfBath'])


## FullBath y HalfBath

FullBath: Full bathrooms above grade

HalfBath: Half baths above grade

In [45]:
cleaned_data['TotalBaths'] = (
    cleaned_data['FullBath'] + (cleaned_data['HalfBath'] * 0.5)
)
cleaned_data = cleaned_data.drop(columns=['FullBath', 'HalfBath'])

## Bedroom

Bedroom: Bedrooms above grade (does NOT include basement bedrooms)


## Kitchen


Kitchen : Kitchens above grade

## KitchenQual

KitchenQual: Kitchen quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor

In [46]:
cleaned_data['KitchenQual'] = cleaned_data['KitchenQual'].replace({
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
})


## TotRmsAbvGrd

TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)


## Functional

Functional: Home functionality (Assume typical unless deductions are warranted)

       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only


In [47]:
cleaned_data['Functional'] = cleaned_data['Functional'].replace({
    'Typ': 7,
    'Min1': 6,
    'Min2': 5,
    'Mod': 4,
    'Maj1': 3,
    'Maj2': 2,
    'Sev': 1,
    'Sal': 0
})


## Fireplaces

Fireplaces: Number of fireplaces

## FireplaceQu


FireplaceQu: Fireplace quality

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace

In [48]:
cleaned_data['FireplaceQu'].fillna('NA', inplace=True)
cleaned_data['FireplaceQu'] = cleaned_data['FireplaceQu'].replace({
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA': 0
})


## GarageType

GarageType: Garage location
		
       2Types	More than one type of garage
       Attchd	Attached to home
       Basment	Basement Garage
       BuiltIn	Built-In (Garage part of house - typically has room above garage)
       CarPort	Car Port
       Detchd	Detached from home
       NA	No Garage

In [49]:
cleaned_data['GarageType'] = cleaned_data['GarageType'].fillna('No Garage')

cleaned_data['GarageType_Reduced'] = cleaned_data['GarageType'].replace({
    'Attchd': 'Attached', 
    'BuiltIn': 'Attached', 
    'Detchd': 'Detached', 
    'Basment': 'Detached', 
    'CarPort': 'Detached', 
    '2Types': 'Other',
    'No Garage': 'No Garage'
})

garage_dummies = pd.get_dummies(cleaned_data['GarageType_Reduced'], prefix='GarageType')

cleaned_data = pd.concat([cleaned_data, garage_dummies], axis=1)
cleaned_data = cleaned_data.drop(columns=['GarageType', 'GarageType_Reduced'])


## GarageYrBlt

GarageYrBlt: Year garage was built


In [50]:
cleaned_data['GarageAge'] = 2024 - cleaned_data['GarageYrBlt']
cleaned_data['GarageAge'] = cleaned_data['GarageAge'].fillna(0)
cleaned_data = cleaned_data.drop(columns=['GarageYrBlt'])



## GarageFinish

GarageFinish: Interior finish of the garage

       Fin	Finished
       RFn	Rough Finished	
       Unf	Unfinished
       NA	No Garage

In [51]:
cleaned_data['GarageFinish'].fillna('NA', inplace=True)
cleaned_data['GarageFinish'] = cleaned_data['GarageFinish'].replace({
    'Fin': 3,
    'RFn': 2,
    'Unf': 1,
    'NA': 0
})

## GarageCars 

GarageCars: Size of garage in car capacity


## GarageArea

GarageArea: Size of garage in square feet


## GarageQual

GarageQual: Garage quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage

In [52]:
cleaned_data['GarageQual'].fillna('NA', inplace=True)
cleaned_data['GarageQual'] = cleaned_data['GarageQual'].replace({
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA': 0
})


## GarageCond

GarageCond: Garage condition

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage

In [53]:
cleaned_data['GarageCond'].fillna('NA', inplace=True)
cleaned_data['GarageCond'] = cleaned_data['GarageCond'].replace({
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA': 0
})


## PavedDrive

PavedDrive: Paved driveway

       Y	Paved 
       P	Partial Pavement
       N	Dirt/Gravel
		

In [54]:
paveddrive_dummies = pd.get_dummies(cleaned_data['PavedDrive'], prefix='PavedDrive')
cleaned_data = pd.concat([cleaned_data, paveddrive_dummies], axis=1)
cleaned_data = cleaned_data.drop(columns=['PavedDrive'])


## WoodDeckSF

WoodDeckSF: Wood deck area in square feet


## OpenPorchSF, EnclosedPorch, 3SsnPorch y  ScreenPorch

OpenPorchSF: Open porch area in square feet

EnclosedPorch: Enclosed porch area in square feet

3SsnPorch: Three season porch area in square feet

ScreenPorch: Screen porch area in square feet


In [55]:
cleaned_data['TotalPorchSF'] = (
    cleaned_data['OpenPorchSF'] + 
    cleaned_data['EnclosedPorch'] + 
    cleaned_data['3SsnPorch'] + 
    cleaned_data['ScreenPorch']
)

cleaned_data = cleaned_data.drop(columns=['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'])


## PoolArea

PoolArea: Pool area in square feet


## PoolQC

PoolQC: Pool quality
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool

In [56]:
cleaned_data = cleaned_data.drop(columns=['PoolQC'])



## Fence

Fence: Fence quality
		
       GdPrv	Good Privacy
       MnPrv	Minimum Privacy
       GdWo	Good Wood
       MnWw	Minimum Wood/Wire
       NA	No Fence

In [57]:
cleaned_data = cleaned_data.drop(columns=['Fence'])


## MiscFeature

MiscFeature: Miscellaneous feature not covered in other categories
		
       Elev	Elevator
       Gar2	2nd Garage (if not described in garage section)
       Othr	Other
       Shed	Shed (over 100 SF)
       TenC	Tennis Court
       NA	None

In [58]:
cleaned_data = cleaned_data.drop(columns=['MiscFeature'])


## MiscVal


MiscVal: $Value of miscellaneous feature


## MoSold

MoSold: Month Sold (MM)


In [59]:
cleaned_data = cleaned_data.drop(columns=['MoSold'])


 ## YrSold

YrSold: Year Sold (YYYY)


In [60]:
cleaned_data['AgeSold'] = 2024 - cleaned_data['YrSold']
cleaned_data = cleaned_data.drop(columns=['YrSold'])


## SaleType

SaleType: Type of sale
		
       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other
		

In [61]:
sale_type_dummies = pd.get_dummies(cleaned_data['SaleType'], prefix='SaleType')
cleaned_data = pd.concat([cleaned_data, sale_type_dummies], axis=1)
cleaned_data = cleaned_data.drop(columns=['SaleType'])


## SaleCondition

SaleCondition: Condition of sale

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)

In [62]:
sale_type_dummies = pd.get_dummies(cleaned_data['SaleCondition'], prefix='Salecondition')
cleaned_data = pd.concat([cleaned_data, sale_type_dummies], axis=1)
cleaned_data = cleaned_data.drop(columns=['SaleCondition'])

In [63]:
cleaned_data

,Id,LotFrontage,LotArea,Neighborhood,OverallQual,OverallCond,RoofMatl,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Salecondition_Abnorml,Salecondition_AdjLand,Salecondition_Alloca,Salecondition_Family,Salecondition_Normal,Salecondition_Partial
0,1461,80.0,9.360741,1,5,6,1,0.0,2,2,...,0,0,0,1,0,0,0,0,1,0
1,1462,81.0,9.565775,1,6,6,1,108.0,2,2,...,0,0,0,1,0,0,0,0,1,0
2,1463,74.0,9.534668,1,5,5,1,0.0,2,2,...,0,0,0,1,0,0,0,0,1,0
3,1464,78.0,9.208238,1,6,6,1,20.0,2,2,...,0,0,0,1,0,0,0,0,1,0
4,1465,43.0,8.518392,2,8,5,1,0.0,3,2,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,21.0,7.568896,0,4,7,1,0.0,2,2,...,0,0,0,1,0,0,0,0,1,0
1455,2916,21.0,7.546974,0,4,5,1,0.0,2,2,...,0,0,0,1,1,0,0,0,0,0
1456,2917,160.0,9.903538,1,5,7,1,0.0,2,2,...,0,0,0,1,1,0,0,0,0,0
1457,2918,62.0,9.253591,1,5,5,1,0.0,2,2,...,0,0,0,1,0,0,0,0,1,0


In [64]:
summary = [
    [col, cleaned_data[col].dtype, cleaned_data[col].count(), cleaned_data[col].nunique(), cleaned_data[col].isnull().sum(), cleaned_data.duplicated().sum()]
    for col in cleaned_data.columns
]

df_check = pd.DataFrame(summary, columns=['column', 'dtype', 'instances', 'unique', 'missing_vals', 'duplicates'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

df_check


,column,dtype,instances,unique,missing_vals,duplicates
0,Id,int64,1459,1459,0,0
1,LotFrontage,float64,1459,116,0,0
2,LotArea,float64,1459,1106,0,0
3,Neighborhood,int64,1459,3,0,0
4,OverallQual,int64,1459,10,0,0
5,OverallCond,int64,1459,9,0,0
6,RoofMatl,int64,1459,2,0,0
7,MasVnrArea,float64,1459,303,0,0
8,ExterQual,int64,1459,4,0,0
9,ExterCond,int64,1459,5,0,0


In [65]:
cleaned_data.fillna(0, inplace=True)


In [66]:
cleaned_data = cleaned_data.drop(columns=['Fireplaces'])
cleaned_data = cleaned_data.drop(columns=['GarageCars'])
cleaned_data = cleaned_data.drop(columns=['GarageCond'])
cleaned_data = cleaned_data.drop(columns=['MSZoning_RM'])
cleaned_data = cleaned_data.drop(columns=['GarageType_Attached'])
cleaned_data = cleaned_data.drop(columns=['GarageType_No Garage'])
cleaned_data = cleaned_data.drop(columns=['PavedDrive_N'])
cleaned_data = cleaned_data.drop(columns=['Salecondition_Partial'])


In [67]:
#Ruta para guardar el archivo CSV
csv_path = "Data/test_2.csv"

# Guardar el DataFrame como un archivo CSV
cleaned_data.to_csv(csv_path, index=False)#

print(f"El dataset se ha guardado en {csv_path}")

El dataset se ha guardado en Data/test_2.csv


In [68]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 89 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Id                              1459 non-null   int64  
 1   LotFrontage                     1459 non-null   float64
 2   LotArea                         1459 non-null   float64
 3   Neighborhood                    1459 non-null   int64  
 4   OverallQual                     1459 non-null   int64  
 5   OverallCond                     1459 non-null   int64  
 6   RoofMatl                        1459 non-null   int64  
 7   MasVnrArea                      1459 non-null   float64
 8   ExterQual                       1459 non-null   int64  
 9   ExterCond                       1459 non-null   int64  
 10  BsmtQual                        1459 non-null   int64  
 11  BsmtCond                        1459 non-null   int64  
 12  BsmtExposure                    14